<a href="https://colab.research.google.com/github/MayyarAA/NaturalLangaugeProcessingBERT/blob/main/MSCI598ProjectBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [3]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 11.8 MB/s 
     |████████████████████████████████| 6.5 MB 35.6 MB/s 
     |████████████████████████████████| 895 kB 28.4 MB/s 
     |████████████████████████████████| 77 kB 4.8 MB/s 
     |████████████████████████████████| 596 kB 35.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
import matplotlib.pyplot as plt
import sklearn    
from sklearn.model_selection import train_test_split

import transformers
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [25]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
classifier("Great! I'm so happy this exists.")

[{'label': 'POSITIVE', 'score': 0.9998798370361328}]

In [ ]:
classifier("I was so bad I can't tell!")

[{'label': 'NEGATIVE', 'score': 0.9997102618217468}]

In [5]:
from google.colab import drive
drive.mount('/content/drive')     


Mounted at /content/drive


In [15]:
train_body = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_bodies.csv")

In [17]:
train_body = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_bodies.csv")
train_stance = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train_stances.csv")

In [18]:
train_body.head(5)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [19]:
train_stance.head(5)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [35]:
train_stance.loc[train_stance['Body ID']==712].head(2)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1787,Seth Rogen to Play Apple’s Steve Wozniak,712,discuss


In [28]:
full_sb = pd.merge(train_stance,train_body,on="Body ID")
full_sb.head(1)

,Headline,Body ID,Stance,articleBody
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\n\n...


In [34]:
import re
def removeSpecailCharFromString(orgWord):
    if(len(orgWord)>100):
      orgWord = orgWord[:100];
    orgWord = orgWord.rstrip('\n')
    return re.sub(r"[^a-zA-Z0-9]+", ' ', orgWord).strip()

In [30]:
inputs = tokenizer(full_sb.iloc[0].Headline, return_tensors="pt")
inputs

{'input_ids': tensor([[  101,  2610,  2424,  3742,  9729,  2007,  2012,  2560,  1005,  2321,
          4230,  1005,  2379,  3290,  2237,  2073,  4724,  2493,  5419,  2044,
          2610, 13249,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [31]:
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [ ]:
#data cleaning


#tokenize use BERT

#load model
#fine tune model 